# Sizing of pods' CPU request

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
from stats_service import StatsService

In [2]:
stats_service = StatsService(name="CPU usage")
stats_service.load_time_series(path="data/cpu_usage.json")